# **TechHorizon**
# Task 3: Digit Drawing Recognition Web App
**Internship Task – TechHorizon**

**Student / Intern: Ameer Hamza**



As part of Task 3, the goal is to build a simple web interface that allows users to upload or draw handwritten digits and receive predictions.

We continue from Task 1, where a model was trained to recognize handwritten digits (e.g., using the MNIST dataset) and subsequently saved.

**In this task, we load the previously saved model from Task 1** and use it to make predictions through a basic web interface. This interface allows real-time digit classification without retraining the model.

In [13]:
import gradio as gr
import cv2

In [14]:
import numpy as np
import gradio as gr
from tensorflow import keras
import cv2

MODEL_PATH = "TechHorizonTask1Model.keras"
model = keras.models.load_model(MODEL_PATH)  # Load once at start

def preprocess(img):
    """
    Convert canvas (H,W,3) uint8 image to 28x28 grayscale [0,1].
    """
    if img is None:
        return np.zeros((28,28), dtype=np.float32)

    # If img is a dict, extract 'image'
    if isinstance(img, dict):
        img = img.get("image", None)
        if img is None:
            return np.zeros((28,28), dtype=np.float32)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Threshold to enhance contrast (make strokes darker)
    _, gray = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)

    # Resize with padding to keep aspect ratio
    h, w = gray.shape
    scale = 20.0 / max(h, w)
    resized = cv2.resize(gray, (int(w * scale), int(h * scale)), interpolation=cv2.INTER_AREA)

    canvas = np.full((28, 28), 0, dtype=np.uint8)  # Black background
    y_off = (28 - resized.shape[0]) // 2
    x_off = (28 - resized.shape[1]) // 2
    canvas[y_off:y_off + resized.shape[0], x_off:x_off + resized.shape[1]] = resized

    # Normalize to [0,1]
    norm = canvas.astype("float32") / 255.0

    return norm

def predict_digit(img):
    x = preprocess(img).reshape(1, 28, 28)
    preds = model.predict(x, verbose=0)[0]
    top = int(np.argmax(preds))
    probs = {str(i): float(p) for i, p in enumerate(preds)}
    return top, probs

def clear_canvas():
    return None

with gr.Blocks() as demo:
    gr.Markdown("# ✏️ MNIST Digit Recognizer")
    gr.Markdown("Draw a digit (0–9) in the canvas below and click **Predict**.")
    with gr.Row():
        canvas = gr.ImageEditor(
            sources=["upload"],
            type="numpy",
            image_mode="L",
            width=280, height=280
        )
        with gr.Column():
            btn = gr.Button("Predict")
            clear = gr.Button("Clear")
            label = gr.Label(num_top_classes=10)
            number = gr.Number(label="Predicted Digit", precision=0)
    btn.click(predict_digit, inputs=[canvas], outputs=[number, label])
    clear.click(fn=clear_canvas, inputs=None, outputs=[canvas])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b8df57299be765bc21.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [15]:
with gr.Blocks() as demo:
    gr.Markdown("# 🧠 MNIST Digit Recognizer")
    gr.Markdown("Draw a digit (0–9) below, then click **Predict**.")

    with gr.Row():
        canvas = gr.Sketchpad(label="Draw here", height=280, width=280)
        with gr.Column():
            btn = gr.Button("Predict")
            clear = gr.Button("Clear")
            number = gr.Number(label="Predicted Digit", precision=0)
            label = gr.Label(num_top_classes=10)

    btn.click(predict_digit, inputs=[canvas], outputs=[number, label])
    clear.click(fn=clear_canvas, inputs=None, outputs=[canvas])
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a88e9797969e40b33a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
